## Chapter 3. 神经网络

In [91]:
import jax.numpy as jnp
import jax
import matplotlib.pyplot as plt
import pickle

In [92]:
# activation function
def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))

def relu(x):
    return jnp.maximum(0, x)

def softmax(x):
    c = jnp.max(x)
    exp_x = jnp.exp(a-c)
    sum_exp_x = jnp.sum(exp_x)
    y = exp_x / sum_exp_x

    return y

def softmax_batch(x):
    x = x - jnp.max(x, axis=-1, keepdims=True)   # オーバーフロー対策
    return jnp.exp(x) / jnp.sum(jnp.exp(x), axis=-1, keepdims=True)

# test
x = jnp.array([-1.0, 1.0, 2.0])
print(jnp.allclose(sigmoid(x), jnp.array([ 0.26894142, 0.73105858, 0.88079708])))

a = jnp.array([0.3, 2.9, 4.0])
print(jnp.allclose(jnp.sum(softmax(x)), 1))

True
True


In [93]:
from dataset.mnist import load_mnist
# 第一次调用会花费几分钟......
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
# 输出各个数据的形状 
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000,) 
print(x_test.shape) # (10000, 784) 
print(t_test.shape) # (10000,)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [94]:
import numpy as np
from dataset.mnist import load_mnist 
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img)) 
    display(pil_img)

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label) # 5
print(img.shape) # (784,)
img = img.reshape(28, 28) # 把图像的形状变成原来的尺寸 
print(img.shape) # (28, 28)
img_show(img)

5
(784,)
(28, 28)


In [95]:
def get_data():
    (x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, flatten=True, one_hot_label=False) 
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3'] 
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = jnp.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = jnp.dot(z1, W2) + b2 
    z2 = sigmoid(a2)
    a3 = jnp.dot(z2, W3) + b3
    y = softmax_batch(a3)
    return y

In [96]:

x, t = get_data() 
network = init_network()
batch_size = 100 # 批数量
accuracy_cnt = 0
for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


## Chapter 4. 神经网络的学习

In [97]:
def cross_entropy_error(y, t): 
    delta = 1e-7
    return -jnp.sum(t * jnp.log(y + delta))

def cross_entropy_error_batch(y, t): 
    if y.ndim == 1:
        t = t.reshape(1, t.size) 
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * jnp.log(y + 1e-7)) / batch_size

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = jnp.zeros_like(x) # 生成和x形状相同的数组

    for idx in range(x.size): 
        tmp_val = x[idx]
        # f(x+h)的计算
        x.at[idx].set(tmp_val + h)
        fxh1 = f(x)

        # f(x-h)的计算
        x.at[idx].set(tmp_val - h)
        fxh2 = f(x)
        grad.at[idx].set((fxh1 - fxh2) / (2*h))
        x[idx] = tmp_val # 还原值
        
    return grad

key = jax.random.PRNGKey(0)

In [98]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 初始化权重 
        self.params = {} 
        self.params['W1'] = weight_init_std * jax.random.normal(key, shape=(input_size, hidden_size))
        self.params['b1'] = jnp.zeros(hidden_size)
        self.params['W2'] = weight_init_std * jax.random.normal(key, shape=(hidden_size, output_size))
        self.params['b2'] = jnp.zeros(output_size)
    
    def predict(self, x):
        b1, b2 = self.params['b1'], self.params['b2']
        W1, W2 = self.params['W1'], self.params['W2']
        a1 = jnp.dot(x, W1) + b1 
        z1 = sigmoid(a1)
        a2 = jnp.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    
    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error_batch(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = jnp.argmax(y, axis=1) 
        t = jnp.argmax(t, axis=1)
        accuracy = jnp.sum(y == t) / float(x.shape[0]) 
        return accuracy
    

    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) 
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) 
        grads['W2'] = numerical_gradient(loss_W, self.params['W2']) 
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads



In [99]:
# mini-batch
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label = True)
train_loss_list = []

# 超参数
iters_num = 10000
train_size = x_train.shape[0] 
batch_size = 100 
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num): 
    # 获取mini-batch
    # batch_mask = np.random.choice(train_size, batch_size) 
    batch_mask = jax.random.choice(key, train_size, shape=(batch_size,), replace=False)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 计算梯度
    grad = network.numerical_gradient(x_batch, t_batch) 
    # grad = network.gradient(x_batch, t_batch) # 高速版!
    
    # 更新参数
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 记录学习过程
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

TypeError: mul got incompatible shapes for broadcasting: (1, 1000), (1, 3).